In [1]:
import os
import json
import numpy as np
import math
import torch
from tqdm import trange
import models.simplevit as simplevit
import models.unet as unet
import utils.funcs as funcs
import utils.norm as norm
import utils.metrics as metrics

In [2]:
h,w=2025,1350
ks=15
skip=ks
cl=funcs.chunk_list(h,w,ks,skip)
output_channel = 9
thrd=0.5
batch_num=10
c21_thrd, c22_thrd=1.0, 1.1

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

# 分类模型加载
model_simplevit = simplevit.SimpleViT()
checkpoint_simplevit = torch.load('./weights/simpleViT_1.pth')
model_simplevit.load_state_dict(checkpoint_simplevit['sd'])
model_simplevit.cuda()
model_simplevit.eval()

# 分割模型加载
model_unet = unet.UNet()
checkpoint_unet = torch.load('./weights/unet_5.pth')
model_unet.load_state_dict(checkpoint_unet['sd'])
model_unet.cuda()
model_unet.eval()

device cuda:0


UNet(
  (conv1): conv_block(
    (conv): Sequential(
      (0): Conv2d(9, 32, kernel_size=(1, 1), stride=(1, 1))
      (1): GroupNorm(2, 32, eps=1e-05, affine=True)
      (2): ReLU(inplace=True)
    )
  )
  (conv2): conv_block(
    (conv): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): GroupNorm(4, 64, eps=1e-05, affine=True)
      (2): ReLU(inplace=True)
    )
  )
  (conv3): conv_block(
    (conv): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): GroupNorm(8, 128, eps=1e-05, affine=True)
      (2): ReLU(inplace=True)
    )
  )
  (conv4): conv_block(
    (conv): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): GroupNorm(16, 256, eps=1e-05, affine=True)
      (2): ReLU(inplace=True)
    )
  )
  (conv_u): conv_block(
    (conv): Sequential(
      (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Gro

In [5]:
img_path = "D:/data/20220011140.npz"
data = np.load(img_path)
img = data['arr_0'][:, 0:h, 0:w]
# origin_img = img[output_channel]


# 切成子图
sub_img = []
for c in cl:
    d=img[:, c[0]:c[1],c[2]:c[3]]
    sub_img.append(d)
sub_img = np.asarray(sub_img)


# 分类
x_classify = sub_img[:, :9]
x_classify = norm.nor2(x_classify, 'classify')
cnt = math.ceil(x_classify.shape[0]/batch_num)
new_x_classify = np.array_split(x_classify, cnt, axis=0)

output_classify = []

with torch.no_grad():
    for i in trange(len(new_x_classify)):
        input_classify = new_x_classify[i]
        input_classify = torch.Tensor(input_classify).cuda()

        pred_classify = model_simplevit(input_classify)
        pred_classify=pred_classify.detach().cpu().numpy()
        pred_classify[pred_classify<=thrd]=0 
        pred_classify[pred_classify>thrd]=1

        output_classify.extend(pred_classify)

output_classify = np.asarray(output_classify)
print(output_classify.shape)

100%|██████████| 1215/1215 [06:43<00:00,  3.01it/s]

(12150, 1)


In [6]:
# 分割
indices_seg = np.where(output_classify == 1)[0] # 只对火点子图进行分割
x_seg = sub_img[indices_seg][:, :9]
y_seg = sub_img[indices_seg][:, -1]
y_seg[y_seg<80]=0
y_seg[y_seg>=80]=1
y_seg = np.expand_dims(y_seg, axis=1)

output_seg = []

with torch.no_grad():
    input_seg = norm.nor2(x_seg, 'seg')
    input_seg = torch.Tensor(input_seg).cuda()
            
    pred_seg = model_unet(input_seg)
    pred_seg=pred_seg.detach().cpu().numpy()
    pred_seg[pred_seg<=thrd]=0
    pred_seg[pred_seg>thrd]=1
    output_seg.extend(pred_seg)
output_seg = np.asarray(output_seg)

In [7]:
cnt_fire = 0
tp, fp, tn, fn=0, 0, 0, 0

# 还原图像
# 创建一个全零数组，形状与原始图像相同
restored_img = np.zeros((h,w))

# 将 sub_img 中的子图像放回原始图像对应的位置
for i, c in enumerate(cl):
    if i in indices_seg:
        restored_sub = output_seg[cnt_fire].reshape((15, 15))
        cnt_fire += 1
    else:
        restored_sub = np.zeros((15, 15))
    restored_img[c[0]:c[1], c[2]:c[3]] = restored_sub

In [15]:
img_path = "D:/data/20220011140.npz"
data = np.load(img_path)
img = data['arr_0'][:, 0:h, 0:w]
print(img.shape)
con = img[-1, :]
con[con < 80] = 0
con[con >= 80] = 1
print(con.shape)
print(restored_img.shape)
tp,fp,tn,fn= metrics.cal_hit(restored_img,con)
f1, p, r, iou, acc = metrics.cal_score(tp, fp, tn, fn)
print(f1, p, r, iou, acc)

(11, 2025, 1350)
(2025, 1350)
(2025, 1350)
0.8641975308641975 0.8085808580858086 0.928030303030303 0.7608695652173912 0.999943667123914


In [8]:
data_path = './processor/final/data/'
img_file = '20220041350.npz'
img_id = os.path.splitext(img_file)[0]
img_path = os.path.join(data_path, img_file)
data = np.load(img_path)
img = data['arr_0'][:, 0:h, 0:w]
# origin_img = img[output_channel]


# 切成子图
sub_img = []
for c in cl:
    d=img[:, c[0]:c[1],c[2]:c[3]]
    sub_img.append(d)
sub_img = np.asarray(sub_img)

# 分类
# x_cls = sub_img[:, :9]
# output_cls = []

# # 阈值初筛
# for i in range(sub_img.shape[0]):
#     p_c21, p_c22 = np.max(sub_img[i, 4]), np.max(sub_img[i, 5])
#     if not (p_c21 >= c21_thrd and p_c22 >= c22_thrd):
#         output_cls.extend([0])
#     else:
#         output_cls.extend([1])

# output_cls = np.asarray(output_cls)
# print(output_cls.shape)

# # 送入分类模型进行进一步筛选
# indices_cls = np.where(output_cls == 1)[0]
# print(indices_cls.shape)
# new_tmp = norm.nor2(x_cls[indices_cls], 'classify')
# cnt = math.ceil(new_tmp.shape[0]/batch_num)
# new_x_cls = np.array_split(new_tmp, cnt, axis=0)
# output = []

# with torch.no_grad():
#     for i in trange(len(new_x_cls)):
#         input_cls = new_x_cls[i]
#         input_cls = torch.Tensor(input_cls).cuda()

#         pred_cls = model_simplevit(input_cls)
#         pred_cls = pred_cls.detach().cpu().numpy()
#         pred_cls[pred_cls<=thrd]=0
#         pred_cls[pred_cls>thrd]=1
#         output.extend(pred_cls)
# output = np.asarray(output)
# print(output.shape)

# output_cls = output_cls.reshape(-1,1)
# output_cls[indices_cls] = output

# 分割
# indices_seg = np.where(output_cls == 1)[0] # 只对火点子图进行分割
# x_seg = sub_img[indices_seg][:, :9]
# y_seg = sub_img[indices_seg][:, -1]

x_seg = sub_img[:, :9]
new_tmp = norm.nor2(x_seg, 'seg')
cnt = math.ceil(new_tmp.shape[0]/batch_num)
new_x_seg = np.array_split(new_tmp, cnt, axis=0)

y_seg = sub_img[:, -1]
y_seg[y_seg<80]=0
y_seg[y_seg>=80]=1
y_seg = np.expand_dims(y_seg, axis=1)

output_seg = []

with torch.no_grad():
    for i in trange(len(new_x_seg)):
        # input_seg = norm.nor2(x_seg, 'seg')
        input_seg = new_x_seg[i]
        input_seg = torch.Tensor(input_seg).cuda()
                
        pred_seg = model_unet(input_seg)
        pred_seg=pred_seg.detach().cpu().numpy()
        pred_seg[pred_seg<=thrd]=0
        pred_seg[pred_seg>thrd]=1
        output_seg.extend(pred_seg)
output_seg = np.asarray(output_seg)

cnt_fire = 0
tp, fp, tn, fn=0, 0, 0, 0

# 还原图像
# 创建一个全零数组，形状与原始图像相同
restored_img = np.zeros((h,w))

# 将 sub_img 中的子图像放回原始图像对应的位置
# for i, c in enumerate(cl):
#     if i in indices_seg:
#         restored_sub = output_seg[cnt_fire].reshape((15, 15))
#         cnt_fire += 1
#     else:
#         restored_sub = np.zeros((15, 15))
#     restored_img[c[0]:c[1], c[2]:c[3]] = restored_sub
for i, c in enumerate(cl):
    restored_img[c[0]:c[1], c[2]:c[3]] = output_seg[cnt_fire]
    cnt_fire += 1

con = img[-1, :]
con[con < 80] = 0
con[con >= 80] = 1
print(con.shape)
print(restored_img.shape)
tp,fp,tn,fn= metrics.cal_hit(restored_img,con)
f1, p, r, iou, acc = metrics.cal_score(tp, fp, tn, fn)
print(f1, p, r, iou, acc)

# file:20220021045 tp:79 fp:34 tn:15179 fn:8 f1:0.790000 p:0.699115 r:0.908046 iou:0.652893 acc:0.997255

100%|██████████| 1215/1215 [00:18<00:00, 64.64it/s]


(2025, 1350)
(2025, 1350)
0.7937007874015748 0.675603217158177 0.9618320610687023 0.6579634464751958 0.9999520804755373


In [66]:
# data = {
#         "id": img_id,
#         "tp": tp,
#         "fp": fp,
#         "tn": tn,
#         "fn": fn,
#         "f1": f1,
#         "p": p,
#         "r": r,
#         "iou": iou,
#         "acc": acc
#     }

data = {
        "id": 1,
        "tp": 2,
        "fp": 3,
        "tn": 4,
        "fn": 5,
        "f1": 6,
        "p": 7,
        "r": 8,
        "iou": 9,
        "acc": 10
    }


# # 将数据写入 JSON 文件
# with open("score.json", "w") as json_file:
#     json.dump(data, json_file, indent=4)
print(data.id)

AttributeError: 'dict' object has no attribute 'id'

In [58]:
def read_variables(img_id):
    with open('score.json', 'r') as json_file:  # 打开 JSON 文件
        data = json.load(json_file)  # 加载 JSON 数据
        img_info = {}  # 创建一个空字典
        for entry in data:  # 迭代 JSON 数据中的每个条目
            if entry['id'] == img_id:  # 检查条目的 ID 是否与所需的 ID 匹配
                img_info = entry['info']  # 提取相关信息
                break

    return img_info  # 返回字典

img_info = read_variables(img_id)
print(img_info.id)

AttributeError: 'dict' object has no attribute 'id'

In [68]:
def read_variables(img_id):
    with open('./processor/score.txt', 'r') as file:
        lines = file.readlines()
        img_info = {}  # 创建一个空字典
        for line in lines:
            pairs = line.split()
            cnt = 0
            for pair in pairs:
                name, value = pair.split(':')
                if cnt == 0:
                    cnt += 1
                    file_id = value
                else:
                    img_info[name] = float(value)

            if file_id == img_id:
                break
    return img_info  # 返回字典

img_info = read_variables('20220011320')
print(img_info)

{'tp': 169.0, 'fp': 47.0, 'tn': 32404.0, 'fn': 5.0, 'f1': 0.866667, 'p': 0.782407, 'r': 0.971264, 'iou': 0.764706, 'acc': 0.998406}


In [79]:
data = np.load('D:/data/20220011815.npz')
img = data['arr_0'][:, 0:h, 0:w]
con = img[10]
con[con<80]=0
con[con>=80]=1
print(con.max())
count_zero = (con.size - np.count_nonzero(con))
count_one = np.count_nonzero(con)

print("元素值为0的个数:", count_zero)
print("元素值为1的个数:", count_one)
print(100 * (count_one/count_zero))

1.0
元素值为0的个数: 2733675
元素值为1的个数: 75
0.0027435594940876293


In [3]:
import json
img_id='20220011815'
img_id2='20020011850'
tp=66
fp=12
tn=2733675
fn=9
f1=0.88
p=0.92
r=0.86
iou=0.77
acc=0.99

dict_list = []

# 假设img_info是已经定义好的字典
img_info1 = {'id': img_id, 'tp': int(tp), 'fp': int(fp), 'tn': int(tn), 'fn': int(fn),
        'f1': round(f1, 4), 'p': round(p, 4), 'r': round(r, 4), 'iou': round(iou, 4), 'acc': round(acc, 4)}
dict_list.append(img_info1)

img_info2 = {'id': img_id2, 'tp': int(tp), 'fp': int(fp), 'tn': int(tn), 'fn': int(fn),
        'f1': round(f1, 4), 'p': round(p, 4), 'r': round(r, 4), 'iou': round(iou, 4), 'acc': round(acc, 4)}
dict_list.append(img_info2)


# 定义文件名
file_name = 'img_info.json'

# 将字典写入JSON文件
with open(file_name, 'w') as json_file:
    json.dump(dict_list, json_file, indent=4)

In [4]:
import json

def read_img_info(img_id, file_name):
    # 读取JSON文件中的数据
    with open(file_name, 'r') as json_file:
        data = json.load(json_file)
    
    # 根据img_id查找对应的字典数据
    img_info = None
    for item in data:
        if item['id'] == img_id:
            img_info = item
            break
    
    return img_info

# 定义文件名
file_name = 'img_info.json'

# 定义要查找的img_id
target_img_id = '20020011850'

# 读取对应img_id的字典数据
result = read_img_info(target_img_id, file_name)

if result:
    print("找到对应的字典数据：", result)
else:
    print("未找到对应的字典数据。")

找到对应的字典数据： {'id': '20020011850', 'tp': 66, 'fp': 12, 'tn': 2733675, 'fn': 9, 'f1': 0.88, 'p': 0.92, 'r': 0.86, 'iou': 0.77, 'acc': 0.99}


In [2]:
import numpy as np
data_path = './processor/final/results/20220061335.png'
img = np.load(data_path)

ValueError: Cannot load file containing pickled data when allow_pickle=False